# Сегментация при помощи обнаружения границ

Когда мы смотрим на картинку, то в большинстве случаев можем разделить объекты между собой. А всё потому, что между ними есть границы (контуры). 

Именно это мы можем использовать для сегментации. Так как границы (края) помогают определить форму объектов. 

Но как определить эти края? Здесь в игру вступают фильтры и свёртки. 

> Если не совсем понятно про свёртки, то можно посмотреть [сюда](https://www.analyticsvidhya.com/blog/2017/06/architecture-of-convolutional-neural-networks-simplified-demystified/?utm_source=blog&utm_medium=image-segmentation-article)

## Что такое свёртка? 

Ниже представлена визуализация свёрки, идущей по изображению

<p align="center">
    <img src="https://github.com/serykhelena/PYGuides/blob/main/notebooks/assets/conv_filter.gif?raw=true" alt="drawing" width="800"/>
</p>

Как же это работает? 

1. Берём матрицу весов (ядро свёртки)
2. Кладём её в начало картинки 
3. Выполняем поэлементное умножение 
4. Записываем результат умножения в новую матрицу 
5. Перемещаем матрицу весов на выбранную дельту (страйд) 
6. Выполняем пункты 3 и 4 пока не пройдём по всем пикселям в картинке 

Значения матрицы весов определяют свёртку, то есть эти веса помогают вытаскивать из изображения особенности (фичи). 

Исследователи поняли, что выбранные определённым образом веса помогают обнаруживать горизонтальные или вертикальные границы (или даже комбинации горизонтальных и вертикальных границ). 

## Оператор Собеля

Одной из таких "специальных" весовых матриц является оператор Собеля. Он обычно используется для обнаружения границ на изображении. 

Оператор Собеля состоит из двух весовых матриц: 
* одна для обнаружения горизонтальных границ 
* вторая для обнаружения вертикальных границ 

Давайте посмотрим, как выглядят эти матрицы 

$$ \text{Sobel Filter (Horizontal)} = \begin{bmatrix} 1 & 2 & 1\\ 0 & 0 & 0\\ -1 & -2 & -1 \end{bmatrix} $$

$$ \text{Sobel Filter (Vertical)} = \begin{bmatrix} -1 & 0 & 1\\ -2 & 0 & 2\\ -1 & 0 & 1 \end{bmatrix} $$

Обнаружение границ работает путём сворачивания этих фильтров по изображению. 

Давайте попробуем.


In [ ]:
import os 

import cv2 
import matplotlib.pyplot as plt 
import numpy as np

from skimage.color import rgb2gray
from scipy import ndimage

In [ ]:
fpath = os.path.join(os.path.dirname(os.path.abspath(os.pardir)), "assets", "cv_19.png")
# fpath = os.path.join(os.pardir, "assets", "cv_19.png")
original_image = cv2.imread(fpath)
original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=[6, 6])
plt.imshow(original_image)
plt.show()

Как видно, на картинке есть много горизонтальных и вертикальных линий. 

Давайте сконвертируем изображение в оттенки серого и применим оператор Собеля (комбинированный, т.е. горизонтальные и вертикальные линии).

In [ ]:
gray = rgb2gray(original_image)

# определяем фильтры Собеля
sobel_horizontal = np.array([np.array([1, 2, 1]), np.array([0, 0, 0]), np.array([-1, -2, -1])])
print(f"Ядро для обнаружения горизонтальных границ\n {sobel_horizontal}\n")
 
sobel_vertical = np.array([np.array([-1, 0, 1]), np.array([-2, 0, 2]), np.array([-1, 0, 1])])
print(f"Ядро для обнаружения вертикальных границ\n {sobel_vertical}")

In [ ]:
out_h = ndimage.convolve(gray, sobel_horizontal, mode='reflect')
out_v = ndimage.convolve(gray, sobel_vertical, mode='reflect')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=[10, 6])

ax[0].imshow(out_h, cmap="gray")
ax[0].set_title("Горизонтальные границы")

ax[1].imshow(out_v, cmap="gray")
ax[1].set_title("Вертикальные границы")

plt.show()

## Оператор Лапласа

Есть ещё один оператор, который умеет находить как горизонтальные, так и вертикальные границы. 

$$ \text{Laplace Operator} = \begin{bmatrix} 1 & 1 & 1\\ 1 & -8 & 1\\ 1 & 1 & 1 \end{bmatrix} $$

Давайте и его попробуем.

In [ ]:
kernel_laplace = np.array([np.array([1, 1, 1]), np.array([1, -8, 1]), np.array([1, 1, 1])])
print(f"Ядро Лапласа\n{kernel_laplace}")

In [ ]:
out_l = ndimage.convolve(gray, kernel_laplace, mode='reflect')

plt.figure(figsize=[6, 6])
plt.imshow(out_l, cmap="gray")
plt.title("Оператор Лапласа")
plt.show()

Пам-пам! оператор Лапласа нашёл как горизонтальные, так и вертикальные границы. 

`Есть некоторая сущность под названием детектор границ Кенни. Когда-нибудь он обязательно будет описан здесь, а пока` [он есть тут](https://habr.com/ru/post/114589/), [тут](https://robocraft.ru/blog/computervision/484.html)

## Заключение 

В этот раз мы посмотрели как находить границы на изображении. Результаты немного отличаются от масок, которые обычно используются в задачах сегментации. Но тем не менее, в некоторых задачах определения вертикальных и горизонтальных линий может быть достаточно для сегментации. 

Например, представим, что есть скан документа и на нём нужно найти таблицу. Тут то и приходятся те операторы, которые были рассмотрены сегодня. 

## Полезные ссылки 

* [Introduction to Image Segmentation Techniques](https://www.analyticsvidhya.com/blog/2019/04/introduction-image-segmentation-techniques-python/)